In [46]:
%pylab inline

import os
import gc
import pdet
import pltutil
import numpy as np
from fpfs import simutil
import scipy.ndimage as ndi
import matplotlib.pylab as plt
import astropy.io.fits as pyfits
from fpfs.imgutil import gauss_kernel
wDir=os.path.join(os.environ['homeWrk'],'FPFS2')

Populating the interactive namespace from numpy and matplotlib


In [7]:
ishear=1
igroup=-1
ngrid=64
if igroup>=0:
    ngal=100
    img1=pyfits.getdata(os.path.join(wDir,'sim/galaxy_basicCenter_psf60/image-%d-g%d-0000.fits' %(igroup,ishear)))
    img2=pyfits.getdata(os.path.join(wDir,'sim/galaxy_basicCenter_psf60/image-%d-g%d-2222.fits' %(igroup,ishear)))
else:
    ngal=1
    img1=simutil.make_basic_sim('basicCenter_psf60','g%d-0000' %ishear,0,ny=ngal,nx=ngal,do_write=False)
    img2=simutil.make_basic_sim('basicCenter_psf60','g%d-2222' %ishear,0,ny=ngal,nx=ngal,do_write=False)
ngrid2=ngrid*ngal

In [118]:
# PSF
psf    =   pyfits.getdata(os.path.join(wDir,'sim/galaxy_basic_psf60/psf-60.fits'))
npad   =   (ngrid2-psf.shape[0])//2
psfData=   np.pad(psf,(npad+1,npad),mode='constant')
assert psfData.shape[0]==ngrid2
psfF   =   np.fft.fftshift(np.fft.fft2(psfData))
gsigma =   3.*2.*np.pi/64

In [126]:
# all of the functions in Fourier space 
# are centered at stamp's centroid
gKer,(k2grid,k1grid)=gauss_kernel(ngrid2,ngrid2,gsigma,do_shift=True,return_grid=True)
q1Ker  =   (k1grid**2.-k2grid**2.)/gsigma**2.*gKer
q2Ker  =   (2.*k1grid*k2grid)/gsigma**2.*gKer
d1Ker  =   (-1j*k1grid)*gKer
d2Ker  =   (-1j*k2grid)*gKer
for (j,i) in pdet._default_inds:
    y      =   j-2
    x      =   i-2
    r1     =   (q1Ker+x*d1Ker-y*d2Ker)*np.exp(1j*(k1grid*x+k2grid*y))
    r2     =   (q2Ker+y*d1Ker+x*d2Ker)*np.exp(1j*(k1grid*x+k2grid*y))
    imgF   =   np.fft.fftshift(np.fft.fft2(img1))
    print(np.sum(r2*imgF/psfF).real/ngrid/ngrid)
    r1t    =   q1Ker+x*d1Ker-y*d2Ker
    r2t    =   q2Ker+y*d1Ker+x*d2Ker
    imgR1  =   np.fft.ifft2(np.fft.ifftshift(imgF/psfF*r1t))
    imgR2  =   np.fft.ifft2(np.fft.ifftshift(imgF/psfF*r2t))
    print(imgR2[y%ngrid,x%ngrid].real)
    coords   = np.array(np.zeros(1),dtype=[('pdet_y','i4'),('pdet_x','i4')])
    indX=np.arange(32,ngal*64,64)
    indY=np.arange(32,ngal*64,64)
    inds=np.meshgrid(indY,indX,indexing='ij')
    coords['pdet_y']=np.ravel(inds[0])
    coords['pdet_x']=np.ravel(inds[1])
    out1=pdet.get_shear_response(img1,psfData,gsigma=gsigma,coords=coords)
    print(out1['pdet_v%d%dr2' %(j,i)])
    gc.collect()

-0.004242182751999185
-0.004242182751999184
[-0.00424218]
-0.005135049865181934
-0.005135049865181934
[-0.00513505]
-0.00524920935395393
-0.005249209353953926
[-0.00524921]
-0.005140092699330462
-0.005140092699330463
[-0.00514009]
-0.005550972680337371
-0.005550972680337369
[-0.00555097]
-0.0051400927036513765
-0.005140092703651376
[-0.00514009]
-0.005249209348346341
-0.005249209348346339
[-0.00524921]
-0.005135049862548624
-0.0051350498625486254
[-0.00513505]
-0.0042421827518683355
-0.004242182751868335
[-0.00424218]


In [109]:
ny,nx   =   psfData.shape
psfF2   =   np.fft.fft2(np.fft.ifftshift(psfData))

gKer2,(k2grid2,k1grid2)=gauss_kernel(ny,nx,gsigma,return_grid=True)

# convolved images
imgF2    =   np.fft.fft2(img1)/psfF2*gKer2
del psfF2,psfData
imgCov  =   np.fft.ifft2(imgF2).real
# Q
imgFQ1  =   imgF2*(k1grid2**2.-k2grid2**2.)/gsigma**2.
imgFQ2  =   imgF2*(2.*k1grid2*k2grid2)/gsigma**2.
imgCovQ1=   np.fft.ifft2(imgFQ1).real
imgCovQ2=   np.fft.ifft2(imgFQ2).real
del imgFQ1,imgFQ2 # these images take a lot of memory

# D
imgFD1  =   imgF2*(-1j*k1grid2)
imgFD2  =   imgF2*(-1j*k2grid2)
imgCovD1=   np.fft.ifft2(imgFD1).real
imgCovD2=   np.fft.ifft2(imgFD2).real
gc.collect()

46